In [ ]:
# default_exp longevity10k

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from proseflow.load import load

In [ ]:
import pandas as pd
import io
import requests
def get_content(url, decode='utf-8'):
    s = requests.get(url).content
    if decode:
        return io.StringIO(s.decode('utf-8'))
    return s

In [ ]:
#url_content("https://raw.githubusercontent.com/junwang4/causal-language-use-in-science/master/data/pubmed_causal_language_use.csv")

In [ ]:
#TODO ->load()
papers = pd.read_csv("data/pubmed_longevity10k.csv")
#papers = load("data/pubmed_longevity10k.csv")

In [ ]:
cause_dataset="https://raw.githubusercontent.com/junwang4/causal-language-use-in-science/master/data/pubmed_causal_language_use.csv"

In [ ]:
df=pd.read_csv(get_content(cause_dataset))
df.columns = ["text", "labels"]

#make multilabel into binary!
df =pd.DataFrame({"text": df["text"], "labels": df["labels"].map(lambda l: 0 if l==0 else 1)})

# Label = 0 : No relationship (1356 cases)
# Label = 1 : Direct causal (494 cases)
# Label = 2 : Conditional causal (213 cases)
# Label = 3 : Correlational (998 cases) 

,text,label
0,Levels of cholesterol fractions in patients wi...,0
1,Faster aspart and IAsp were confirmed noninfer...,0
2,Major operative morbidity after minimally inva...,0
3,"The promise of combining risk assessment, comm...",0
4,PPARÎ´ peroxisome proliferator-activated recep...,0
...,...,...
3056,The etiology of anemia appears to be iron-rela...,3
3057,DM is associated with poor outcomes in patient...,3
3058,The BDI is a significant predictor of long-ter...,3
3059,Poor glycemic control among diabetics is a ris...,3


In [ ]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.2)

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel, ClassificationModel,ClassificationArgs


In [ ]:
import torch
torch.device('cpu')

train_df, eval_df = train_test_split(df, test_size=0.2)

device(type='cpu')

In [ ]:
model = ClassificationModel("roberta", "roberta-base", args=ClassificationArgs(), use_cuda=False)
#model = MultiLabelClassificationModel("roberta", "roberta-base", args=ClassificationArgs(), num_labels=4, use_cuda=False)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
model.train_model(train_df[:100])

(13, 0.6963304143685561)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

/Users/markus/Library/Caches/pypoetry/virtualenvs/proseflow-GKtXBSGs-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [ ]:
#import en_ner_bionlp13cg_md
nlp = load("en_ner_bionlp13cg_md", use_gpu=False)
#nlp = spacy.load("en_ner_bionlp13cg_md")

{'re': '(zh|da|nl|en|fr|de|el|it|ja|lt|nb|pl|pt|ro|es|xx)[_(core|ent|ner)_(web|news|wiki|sci|craft|jnlpba|bc5cdr|bionlp13cg)_(sm|md|lg)]*$'} shape {'use_gpu': False}
Loading SpaCy...


In [ ]:
[(ent.label_, ent) for ent in doc.ents]

[('CELL', cells),
 ('CELL', MDSC),
 ('CELL', myeloid cells),
 ('ORGANISM', mice),
 ('ORGANISM', humans ),
 ('CANCER', cancer),
 ('CANCER',
  hepatocellular 
  carcinoma),
 ('CANCER', HCC)]